In [3]:
####################################################
#
# T E S T   D A T A   G E N E R A T I O N
#
# this code is not part of mendonca algorithm
# and used only for experiment purpose in
# order to generate cameras and points
# 
####################################################

In [4]:
import numpy as np
import cv2
import scipy.optimize as opt
import math
import matplotlib.pyplot as plt
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D

np.set_printoptions(suppress=True)

In [5]:
state = pd.read_csv("state.csv")

In [6]:
rng = np.random.default_rng(3122787423)

In [7]:
def p3t(T,x,y,z):
    # apply a Projective 3D Transform
    xyz = np.concatenate((x, y, z),axis=1)
    column_ones = np.ones((len(x),1))
    tmp = T @ (np.concatenate((xyz,column_ones),axis=1)).T
    xp = (tmp[0,:]/tmp[3,:]).T
    yp = (tmp[1,:]/tmp[3,:]).T
    zp = (tmp[2,:]/tmp[3,:]).T
    return xp,yp,zp

In [8]:
def projf(P,x,y,z):
#     % PROJ  compute perspective projection (from 3D to pixel coordinates)
#     %   pixel positions are returned with floating point precision
#     %
#     %   See also PROJE

    c3d = np.concatenate((x, y, z),axis=1)
    column_ones = np.ones((len(x),1))
    h3d = (np.concatenate((c3d,column_ones),axis=1)).T
    h2d = P @ h3d

    c2d = h2d/ h2d[2,:]

    u = c2d[0,:].T
    v = c2d[1,:].T
    return u,v

In [9]:
def plot_3d(x,y,z):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(x,y,z)
    plt.show()

In [10]:
def plot_2d(x,y):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.scatter(x,y)
    plt.show()

In [11]:
def conver_to_col(x):
    return np.reshape(x,(len(x),1))

In [12]:
def rcam(A):
# RCAM generate a random camera
#    generate a random camera pointing to lookp, positioned at an average 
#    distance ad form the origin, with a std dev of sd 
#    A is the intrinsic parameters matrix

    ad=2.5
    sd=0.25
    lookp=np.zeros((1,3))
    eyep = rng.uniform(-1,1,size=(1,3))-0.5
    R = np.zeros((3,3))
    eyep = eyep/np.linalg.norm(eyep) * (ad + sd*rng.random(1))

    R[2,:] = lookp - eyep/np.linalg.norm(lookp - eyep)
    R[1,:] = np.cross(R[2,:],rng.uniform(size=(1,3)))
    R[1,:] = R[1,:]/np.linalg.norm(R[1,:])
    R[0,:] = np.cross(R[1,:],R[2,:])
    Rt = np.concatenate((R,-R @ eyep.T),axis=1)

    return A @ Rt


In [13]:
numberOfViews = 5
numberOfPoints = 50
imagePoints = np.zeros((numberOfPoints,2,numberOfViews))
PPM = np.zeros((3,4,numberOfViews))
PPMGT = np.zeros((3,4,numberOfViews))

In [14]:
def create_fake_points():
    data = (rng.uniform(size=(numberOfPoints,3),)-0.5)/(math.sqrt(3)/2)
    x = data[:,0]
    y = data[:,1]
    z = data[:,2]
    return x, y, z

In [15]:
true_K = np.array([
    [800,0,256],
    [0,800,256],
    [0,0,1]
    ])
x, y, z = create_fake_points()
A = true_K
P = rcam(A)
invAP = np.linalg.inv(A) @ P
lower_line = [[0,0,0,1]]
G0 = np.concatenate((invAP, lower_line),axis=0)

In [16]:
x,y,z = p3t(
    G0,
    conver_to_col(x),
    conver_to_col(y),
    conver_to_col(z)
)
P = A @ np.concatenate((np.identity(3), np.zeros((3,1))),axis=1)

u,v = projf(P,conver_to_col(x),conver_to_col(y),conver_to_col(z))

imagePoints[:,0,0] = u
imagePoints[:,1,0] = v
PPMGT[:,:,0] = P

In [17]:
for view in range (1,numberOfViews):
    # random camera position
    P = rcam(A)
    # apply world coordinate transformation
    P = P @ np.linalg.inv(G0);
    # project world points to image points
    u,v = projf(P,conver_to_col(x),conver_to_col(y),conver_to_col(z))
    imagePoints[:,0,view] = u
    imagePoints[:,1,view] = v
    PPMGT[:,:,view] = P   

# A Simple Technique for Self-Calibration
Alexander Kruglyak

Sofya Zybtsovsky

### Introduction
The goal of Computer Vision is to compute properties (mainly geometric) of the three-dimensional world from images.
One of the challenging problems of Computer Vision is to reconstruct a 3D model of a scene from a moving camera.
<br>Possible applications include: navigation of autonomous vehicles, objects recognition, reverse engineering and synthesis of virtual environments.


In general assumption that the intrinsic parameters of the camera (focal length, image center and aspect ratio) are known. <br>
Computing camera motion in this case is a well known problem in photogrammetry, called relative orientation, for which several methods are available.<br>Given all the parameters of the camera, reconstruction is straightforward.


## Camera matrix
Camera matrix or (camera) projection matrix is a 3X4 matrix which describes the mapping of a pinhole camera from 3D points in the world to 2D points in an image.

<center><img src="images/ProjectionMatrix.jpg"></center>

## Self-calibration
Self-calibration is the process of finding intrinsic parameters of the camera without any actual calibration object:
- focal length
- image center (principal point)
- aspect ratio (pixel skew)

## Camera calibration matrix K
5 degrees of freedom: 2 for focal length, 2 for offset, and 1 for skewness

<center><img src="images/K_matrix.png"></center>

## Approaches to find calibration matrix

### Hartley algorithm
- non-iterative algorithm 
- finds only focal lengths 
- requires at least 8 conjugate pairs points
- “heavy” mathematics calculation (SVD, determinants equation, and many equation, and more...)


### Hartley algorithm - disadvantages
- determining only the focal lengths 
- requires all other internal camera parameters to be known
- heavy computations


### Mendonca - algorithm
- extension of Hartley’s self-calibration technique
- Input: 3+ images such as contain at least 8+ correspondence points
- Output: <b>all</b> intrinsic parameters

### Mendonca - algorithm
Based on the theorem of essential matrix E=[t]<sub>x</sub>R:
<br><br><center>A real matrix E<sub>3x3</sub> can be factorized as product of a nonzero skew-symmetric matrix <b>t</b> (translation) and a rotation matrix <b>R</b> 
<br>if and only if 
<br>E has 2 identical singular values and a zero singular value</center>

### Mendonca - algorithm
E<sub>3x3</sub> =[t]<sub>x</sub>R   <=>    <sup>1</sup>𝞼=<sup>2</sup>𝞼
<br><sup>3</sup>𝞼=0
<br>algo aims to find 2 singular values (other than zero) 
that are as close as possible:
<br><sup>1</sup>𝞼 - <sup>2</sup>𝞼 -> 0
<br>K is intrinsic matrix

### Mendonca algorithm
Input: 3+ images such as each contains at least 8+ correspondence points

Output:  K (intrinsic  parameters)

1. Init some random matrix K<sub>i</sub> for every image 1≤ i ≤ n
2. Calculate fundamental matrix <b>F</b> for each pair of images (for example via 8-point algorithm with data normalization)
3. Calculate essential matrix <b>E</b> for each pair of images (based on F and K)
4. Decompose E and find singular values (SVD)
4. Calculate error (cost function)
6. Minimize error (e.g, via least squares)
7. Then cost function reach global minimum: return K


### 1. Init some random matrix K

initial_K = [true_K[0,0], true_K[0,2],true_K[1,1],true_K[1,2]]

### 2. Calculate fundamental matrix F for each pair of images (for example via 8-point algorithm with data normalization)


In [19]:
def calc_fund_matrix(points1,points2):
    '''
    Takes in 2 arrays of matching points 
    Return Fundamental matrix computes 
    using 8 point algorithm
    '''
#     F, mask = cv2.findFundamentalMat(points1,points2,cv2.FM_8POINT)
    F, mask = cv2.findFundamentalMat(points1,points2)

    return F 

Fs = np.zeros((3,3,numberOfViews,numberOfViews))

for i in range(numberOfViews):
    for j in range(i+1,numberOfViews):
        Fs[:,:,i,j] = calc_fund_matrix(imagePoints[:,:,i],imagePoints[:,:,j])

### 3. Calculate essential matrix E for each pair of images (based on F and K)

In [20]:
def calculate_essential_matrix(K,F):
    return K.T @ F @ K

### 4. Decompose E and find singular values (SVD)

In [21]:
def decompose_singular_values(E):
    _,D,_ = np.linalg.svd(E)
    # Singular Values (3rd value, D[3] is 0 according to theorem)
    r = D[0]
    s = D[1]
    return r, s

### 5. Calculate error (cost function)

<sup>1</sup>𝞼<sub>ij</sub> , <sup>2</sup>𝞼<sub>ij</sub>  - non zero singular values of E<sub>ij</sub> = K<sup>T</sup><sub>i</sub>F<sub>ij</sub>K<sub>j</sub>, such as <sup>1</sup>𝞼<sub>ij</sub> > <sup>2</sup>𝞼<sub>ij</sub>

w<sub>ij</sub> - normalized weight factors

Minimize cost function:
<center><img src="images/cost_function.png"></center>

In [22]:
def mendonca_cost_func(X):
    '''
    computes Mendonca & Cipolla Cost function to find the Optimal Intrinsic Parameters
    Input
    X      - Approximate Values of Intrinsics - 1D array with length 5
    Output
    E    - Computed Cost
    '''

    K = np.array([
        [X[0],0,X[1]],
        [0,X[2],X[3]],
        [0,0,1]
    ])
    cost = 0
    nof_images = numberOfViews
    Den = nof_images*(nof_images-1)/2 

    for i in range(0,nof_images-1):
        for j in range (i+1,nof_images):
            E = calculate_essential_matrix(K, Fs[:,:,i,j])
            r,s = decompose_singular_values(E)
            cost+= (1/Den) * (r - s)/s

    return cost

### 6. Minimize error (e.g, via least squares)

In [23]:
res = opt.minimize(mendonca_cost_func,x0=initial_K, method='Nelder-Mead')

### 7. Then cost function reach global minimum: return K

In [24]:
result_K = np.array([
    [ res.x[0], 0 ,       res.x[1] ],
    [ 0,        res.x[2], res.x[3] ],
    [ 0,        0,        1        ]
])
print (np.matrix(result_K))

[[799.98911309   0.         255.99920977]
 [  0.         799.98828015 255.99955479]
 [  0.           0.           1.        ]]


### Mendonca | Advantages

- simple mathematical calculation

- find all unknown intrinsic camera parameters

- always converges to the global minimum

- has no bias towards any particular image of the sequence

- insensitive to the initialization


In [25]:
# result_K = np.zeros((3,3))


In [26]:
initial_K = [1,200,1,1]
res = opt.minimize(mendonca_cost_func,x0=initial_K, method='Nelder-Mead')
result_K = np.zeros((3,3))
result_K = np.array([[res.x[0],0,res.x[1]],[0,res.x[2],res.x[3]],[0,0,1]])
print (np.matrix(result_K))

[[ 100966.57497141       0.         -273208.24876514]
 [      0.          280794.97424055   -6312.28821548]
 [      0.               0.               1.        ]]


Mendonca, Paulo RS & Roberto Cipolla "A simple technique for self-calibration" 1999,  University of Cambridge, UK